# Query the live aircraft data from Opensky API

This notebook shows how to query live data from the Opensky API.
https://opensky-network.org/apidoc/

In [34]:
# Columns data from the API
columns = ["icao24",
 "callsign",
 "origin_country",
 "time_position",
 "last_contact",
 "longitude",
 "latitude",
 "baro_altitude",
 "on_ground",
 "velocity",
 "true_track",
 "vertical_rate",
 "sensors",
 "geo_altitude",
 "squawk",
 "spi",
 "position_source",
 "time"]

In [35]:
# Import the libraries that will be used in this notebook
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os

## API configuration and query

In [38]:
# Save config information.
url = "https://opensky-network.org/api/states/all"

# Build partial query URL
query_url = f"{url}"

#  Perform a request for data
response = requests.get(query_url).json()

In [39]:
# Time of the query data
response["time"]

1611977770

In [40]:
# Amount of data point of the query data
len(response["states"])

2615

In [41]:
# One element of the array
response["states"][0]

['a89ea5',
 'N6545H  ',
 'United States',
 1611977770,
 1611977770,
 -97.3402,
 35.979,
 1188.72,
 False,
 28.14,
 251.9,
 -0.65,
 None,
 1143,
 '1200',
 False,
 0]

In [42]:
# Map data to the dictorary
aircraft_live_data = [];
for i in range(0, 10): #range(len(response["states"])):
    data = {};
    # Test for null and strip spaces in case not null
    if response["states"][i][0]:
        data["icao24"] = response["states"][i][0].strip()
    else:
        data["icao24"] = response["states"][i][0]
    if response["states"][i][1]:
        data["callsign"] = response["states"][i][1].strip()
    else:
        data["callsign"] = response["states"][i][1]
    if response["states"][i][2]:
        data["origin_country"] = response["states"][i][2].strip()
    else:
        data["origin_country"] = response["states"][i][2]
    data["time_position"] = response["states"][i][3]
    data["last_contact"] = response["states"][i][4]
    data["longitude"] = response["states"][i][5]
    data["latitude"] = response["states"][i][6]
    data["baro_altitude"] = response["states"][i][7]
    data["on_ground"] = response["states"][i][8]
    data["velocity"] = response["states"][i][9]
    data["true_track"] = response["states"][i][10]
    data["vertical_rate"] = response["states"][i][11]
    data["sensors"] = response["states"][i][12]
    data["geo_altitude"] = response["states"][i][13]
    if response["states"][i][14]:
        data["squawk"] = response["states"][i][14].strip()
    else:
        data["squawk"] = response["states"][i][14]
    data["spi"] = response["states"][i][15]
    data["position_source"] = response["states"][i][16]
    data["time"] = response["time"]
    
    aircraft_live_data.append(data)

In [43]:
aircraft_live_data[0]

{'icao24': 'a89ea5',
 'callsign': 'N6545H',
 'origin_country': 'United States',
 'time_position': 1611977770,
 'last_contact': 1611977770,
 'longitude': -97.3402,
 'latitude': 35.979,
 'baro_altitude': 1188.72,
 'on_ground': False,
 'velocity': 28.14,
 'true_track': 251.9,
 'vertical_rate': -0.65,
 'sensors': None,
 'geo_altitude': 1143,
 'squawk': '1200',
 'spi': False,
 'position_source': 0,
 'time': 1611977770}

### Save data to MySQL

In [44]:
import datetime
from datetime import datetime
import mysql.connector
# import database_credentials as dbkeys
import os
import json

with open("/etc/config.json") as config_file:
    config = json.load(config_file)


# Variables:
# database_name = "project_2"  # the name of the target database
# table_name = "aircraft_data"
database_name = "djangowebsite"  # the name of the target database
table_name = "airtrafficapp_aircrafts"



# Create a function to connect to the MYSQL server
def database_connect(hostname, username, password, database=database_name):
    mydb = mysql.connector.connect(
        host=hostname,
        user=username,
        passwd=password,
        database=database
    )
    return mydb



# Return an object containing the MYSQL connection
mydb = database_connect(
    config.get("MYSQL_HOSTNAME"),
    config.get("MYSQL_USERNAME"),
    config.get("MYSQL_PASSWORD")
    )
print(mydb)


# Create the cursor to manipute databases
my_cursor = mydb.cursor()
# Add unique constraint


my_cursor.execute(f"SELECT * FROM {database_name}.{table_name} ORDER BY id DESC LIMIT 1;")
for records in my_cursor:
    print(records)
    print(records[0])


# my_cursor.execute(f"""ALTER TABLE `project_2`.`aircraft_data` 
# ADD COLUMN `time` INT NULL DEFAULT NULL COMMENT '' AFTER `position_source`""")

# Create place holders records to insert into the table
sqlStuff = f"""INSERT INTO {table_name} (icao24, 
                                        callsign,
                                        origin_country,
                                        time_position,
                                        last_contact,
                                        longitude,
                                        latitude,
                                        baro_altitude,
                                        on_ground,
                                        velocity,
                                        true_track,
                                        vertical_rate,
                                        sensors,
                                        geo_altitude,
                                        squawk,
                                        spi,
                                        position_source,
                                        time)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """

(32905259, '4403f4', 'TAY4403', 'Austria', 1611977649, 1611977649, 5.4767, 50.6806, 518.16, 0, 58.57, 225.36, -2.93, None, None, '1155', 0, 0, 1611977700)
32905259


In [45]:
for  ii in range(len(aircraft_live_data)):
            record1 = (
                aircraft_live_data[ii]["icao24"],
                aircraft_live_data[ii]["callsign"],
                aircraft_live_data[ii]["origin_country"],
                aircraft_live_data[ii]["time_position"],
                aircraft_live_data[ii]["last_contact"],
                aircraft_live_data[ii]["longitude"],
                aircraft_live_data[ii]["latitude"],
                aircraft_live_data[ii]["baro_altitude"],
                aircraft_live_data[ii]["on_ground"],
                aircraft_live_data[ii]["velocity"],
                aircraft_live_data[ii]["true_track"],
                aircraft_live_data[ii]["vertical_rate"],
                aircraft_live_data[ii]["sensors"],
                aircraft_live_data[ii]["geo_altitude"],
                aircraft_live_data[ii]["squawk"],
                aircraft_live_data[ii]["spi"],
                aircraft_live_data[ii]["position_source"],
                aircraft_live_data[ii]["time"]
                )

#             print(record1)
                

            my_cursor.execute(sqlStuff, record1)

mydb.commit()

In [46]:
for i in range(len(response["states"])):
    aircraft_live_data = list(range(0,18))
    if response["states"][i][0]:
        aircraft_live_data[0] = response["states"][i][0].strip()
    else:
        aircraft_live_data[0] = response["states"][i][0]
    if response["states"][i][1]:
        aircraft_live_data[1] = response["states"][i][1].strip()
    else:
        aircraft_live_data[1] =  response["states"][i][1]
    if response["states"][i][2]:
        aircraft_live_data[2] =  response["states"][i][2].strip()
    else:
        aircraft_live_data[2] =  response["states"][i][2]
    aircraft_live_data[3] =  response["states"][i][3]
    aircraft_live_data[4] =  response["states"][i][4]
    aircraft_live_data[5] =  response["states"][i][5]
    aircraft_live_data[6] =  response["states"][i][6]
    aircraft_live_data[7] =  response["states"][i][7]
    aircraft_live_data[8] =  response["states"][i][8]
    aircraft_live_data[9] =  response["states"][i][9]
    aircraft_live_data[10] =  response["states"][i][10]
    aircraft_live_data[11] =  response["states"][i][11]
    aircraft_live_data[12] =  response["states"][i][12]
    aircraft_live_data[13] =  response["states"][i][13]
    if response["states"][i][14]:
        aircraft_live_data[14] =  response["states"][i][14].strip()
    else:
        aircraft_live_data[14] =  response["states"][i][14]
    aircraft_live_data[15] =  response["states"][i][15]
    aircraft_live_data[16]  = response["states"][i][16]
    aircraft_live_data[17] = response["time"]

In [12]:
# aircraft_live_data

In [13]:
# (aircraft_live_data[0], 
#  aircraft_live_data[1],
#  aircraft_live_data[2],
#  aircraft_live_data[3],
#  aircraft_live_data[4],
#  aircraft_live_data[5],
#  aircraft_live_data[6],
#  aircraft_live_data[7],
#  aircraft_live_data[8],
#  aircraft_live_data[9],
#  aircraft_live_data[10],
#  aircraft_live_data[11],
#  aircraft_live_data[12],
#  aircraft_live_data[13],
#  aircraft_live_data[14],
#  aircraft_live_data[15],
#  aircraft_live_data[16]
# )

### Query Database

In [14]:
my_cursor.execute(f"SELECT * FROM {database_name}.{table_name} ORDER BY id DESC LIMIT 25;")
list_records = []
for records in my_cursor:
#     print(f"Total records on the table: {records[0]}")
    print(f"-----------------------------------------")
#     print(records)
    list_records.append(records)
    

-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
-----------------------------------------
----------------------------------

In [15]:
type(records)

tuple

In [16]:
# Convert tuples from database to dataframe
df = pd.DataFrame(list_records, columns = [
                                    "id",
                                    "icao24",
                                     "callsign",
                                     "origin_country",
                                     "time_position",
                                     "last_contact",
                                     "longitude",
                                     "latitude",
                                     "baro_altitude",
                                     "on_ground",
                                     "velocity",
                                     "true_track",
                                     "vertical_rate",
                                     "sensors",
                                     "geo_altitude",
                                     "squawk",
                                     "spi",
                                     "position_source",
                                    "time"]
                 ) 

In [17]:
df.set_index('id')

,icao24,callsign,origin_country,time_position,last_contact,longitude,latitude,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,time
id,,,,,,,,,,,,,,,,,,
50282325,7c6b39,JST412,Australia,1.611976e+09,1611976079,152.9540,-29.9723,11887.20,0,233.19,21.75,0.00,None,12496.80,1450,0,0,1611976080
50282324,a3b88e,AAL508,United States,1.611976e+09,1611976080,-106.1640,34.1841,11582.40,0,189.77,277.16,-0.33,None,11711.90,6220,0,0,1611976080
50282323,0d0c07,VTM787,Mexico,1.611976e+09,1611976079,-98.7951,28.6695,7246.62,0,207.14,208.16,-10.73,None,7559.04,6223,0,0,1611976080
50282322,7c6b1c,JST290,Australia,1.611976e+09,1611976080,173.0530,-42.9267,7010.40,0,221.86,41.90,-0.33,None,7208.52,5027,0,0,1611976080
50282321,0d0c02,VIV1242,Mexico,1.611976e+09,1611976079,-116.1420,32.2920,4328.16,0,172.03,290.11,-12.68,None,4351.02,0034,0,0,1611976080
50282320,a2e5de,HGT3142,United States,1.611976e+09,1611976038,-122.9400,45.5311,129.54,0,62.87,323.32,-3.90,None,83.82,None,0,0,1611976080
50282319,7c6b41,JST765,Australia,1.611976e+09,1611976080,148.1600,-34.7108,11277.60,0,256.90,77.74,0.00,None,11818.60,4367,0,0,1611976080
50282318,aa56da,UAL1749,United States,1.611976e+09,1611976030,-125.0910,37.5988,10363.20,0,190.08,267.05,0.00,None,10454.60,1736,0,0,1611976080
50282317,7c6b2f,JST708,Australia,1.611976e+09,1611975890,144.8360,-37.6593,NaN,1,64.31,171.56,NaN,None,NaN,3277,0,0,1611976080


In [18]:
df.to_dict(orient='records')

[{'id': 50282325,
  'icao24': '7c6b39',
  'callsign': 'JST412',
  'origin_country': 'Australia',
  'time_position': 1611976079.0,
  'last_contact': 1611976079,
  'longitude': 152.954,
  'latitude': -29.9723,
  'baro_altitude': 11887.2,
  'on_ground': 0,
  'velocity': 233.19,
  'true_track': 21.75,
  'vertical_rate': 0.0,
  'sensors': None,
  'geo_altitude': 12496.8,
  'squawk': '1450',
  'spi': 0,
  'position_source': 0,
  'time': 1611976080},
 {'id': 50282324,
  'icao24': 'a3b88e',
  'callsign': 'AAL508',
  'origin_country': 'United States',
  'time_position': 1611976079.0,
  'last_contact': 1611976080,
  'longitude': -106.164,
  'latitude': 34.1841,
  'baro_altitude': 11582.4,
  'on_ground': 0,
  'velocity': 189.77,
  'true_track': 277.16,
  'vertical_rate': -0.33,
  'sensors': None,
  'geo_altitude': 11711.9,
  'squawk': '6220',
  'spi': 0,
  'position_source': 0,
  'time': 1611976080},
 {'id': 50282323,
  'icao24': '0d0c07',
  'callsign': 'VTM787',
  'origin_country': 'Mexico',
  '